<a href="https://colab.research.google.com/github/shoma3571/recommend-engine/blob/main/chapter2/2_MatrixFactorization_MovieLens100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>